In [ ]:
import io
import zipfile

from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import pydicom
from pydicom.filebase import DicomBytesIO

from pymedphys.coll import calc_mu_density, get_grid

In [ ]:
zip_filepath = glob('../../../tests/data/logfiles/Trilogy/*')[0]

In [ ]:
with open(zip_filepath, 'rb') as input_file:
    data = io.BytesIO(input_file.read())

In [ ]:
data_zip = zipfile.ZipFile(data)

In [ ]:
data_zip.namelist()

In [ ]:
with data_zip.open('VMAT_TS4_2-field_H&N/RP.VMAT_TS4_2-field.dcm') as fp:
    raw = DicomBytesIO(fp.read())
    dcm = pydicom.dcmread(raw)

In [ ]:
beam_sequence = dcm.BeamSequence[0]

In [ ]:
leaf_boundaries = beam_sequence.BeamLimitingDeviceSequence[2].LeafPositionBoundaries
leaf_widths = np.diff(leaf_boundaries)
leaf_widths

In [ ]:
np.sum(leaf_widths)

In [ ]:
assert beam_sequence.BeamLimitingDeviceSequence[2].NumberOfLeafJawPairs == len(leaf_widths)

num_leaves = len(leaf_widths)

In [ ]:
control_points = beam_sequence.ControlPointSequence

In [ ]:
mlcs = [
    control_point.BeamLimitingDevicePositionSequence[-1].LeafJawPositions
    for control_point in control_points
]

mlcs = [
    np.array([mlc[num_leaves::], -np.array(mlc[0:num_leaves])]).T
    for mlc in mlcs
]

mlcs = np.array(mlcs)
np.shape(mlcs)

In [ ]:
control_points[0].BeamLimitingDevicePositionSequence[1]

In [ ]:
dicom_jaw = control_points[0].BeamLimitingDevicePositionSequence[1].LeafJawPositions

jaw = np.array(dicom_jaw)[-1::-1]
jaw[1] = -jaw[1]

jaw = jaw[None,:] + np.zeros([50,1])

np.shape(jaw)

In [ ]:
total_mu = np.array(dcm.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
total_mu

In [ ]:
final_mu_weight = np.array(beam_sequence.FinalCumulativeMetersetWeight)
final_mu_weight

In [ ]:
mu = [
    total_mu * np.array(control_point.CumulativeMetersetWeight) / final_mu_weight
    for control_point in control_points
]
mu = np.array(mu)
np.shape(mu)

In [ ]:
mu_density = calc_mu_density(mu, mlcs, jaw, leaf_pair_widths=leaf_widths)

In [ ]:
grid = get_grid(leaf_pair_widths=leaf_widths)

In [ ]:
plt.figure(figsize=(10,8.5))
plt.pcolormesh(grid['mlc'][-1::-1], grid['jaw'][-1::-1], mu_density)
plt.colorbar()
plt.xlabel('MLCX direction (mm)')
plt.ylabel('ASYMY travel direction (mm)')

plt.axis('equal')
# plt.gca().invert_yaxis()